In [1]:
import os
import numpy as np
import IPython
import copy
from shutil import copyfile

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base


import sys
sys.dont_write_bytecode=True

PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from influence.inceptionModel import BinaryInceptionModel
from influence.logisticRegressionWithLBFGS import LogisticRegressionWithLBFGS
import influence.experiments
from influence.dataset import DataSet
# from influence.dataset_poisoning import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.iter_attack import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.Progress import *

from utils import dataset_metadatas, experiment_result_metadata_to_FN, FN_to_experiment_result_metadata, get_dataset, get_full_model_graph
from sklearn.linear_model import LogisticRegression

from load_animals import *

from skimage import io

%load_ext autoreload
%autoreload 2

from data_poisoning import data_poisoning

/home/ubuntu/anaconda3/envs/influence_env/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/influence_env/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/influence_env/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# The goal of this function is to research on the full model transferbility.
# The top models we consider are Logistic Regression and SVM.

# Given which dataset to use, num_to_perterb, IF or FC,
# the function finds the results of data poisoning from the ./Experiment_results/Experiment_1/ folder
# and give the transferbility result: the logits of the true label of target test point 

dataset_classes = dataset_metadatas["Dog-Fish"] 
#num_train_ex_per_class, num_test_ex_per_class = 900, 300
use_IF = True
num_to_perterb = 2


In [3]:
# 1. Get the clean inception data_set 
data_sets = get_dataset(dataset_classes)

Loading animals from disk...
../data/dataset_dog-fish_train-900_test-300.npz


In [5]:
img_side = 299
num_channels = 3 
batch_size = 100
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]
weight_decay = 0.001
    
training_dataset_classes = dataset_classes["classes"]
num_classes = len(training_dataset_classes)
full_graph_inception = tf.Graph()
with full_graph_inception.as_default():
    full_model_name = '%s_inception_wd-%s' % ('_'.join(training_dataset_classes), weight_decay)
    full_model_inception = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)
    
full_graph_inceptionResNet = tf.Graph()
with full_graph_inceptionResNet.as_default():
    full_model_name = '%s_inceptionResNet_wd-%s' % ('_'.join(training_dataset_classes), weight_decay)
    full_model_inceptionResNet = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        use_InceptionResNet = True,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)

self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 2048
/home/ubuntu/anaconda3/envs/influence_env/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
inception_features:  Tensor("flatten/Reshape:0", shape=(?, ?), dtype=float32)
x_poison_features:  Tensor("Gather:0", shape=(1, ?), dtype=float32)
t_target_features:  Tensor("Gather_1:0", shape=(1, ?), dtype=float32)
Lp:  Tensor("norm/Squeeze:0", shape=(), dtype=float32)
LP_gradient Tensor("strided_slice_1:0", shape=(268203,), dtype=float32)
self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 1536


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
inception_features:  Tensor("flatten/Reshape:0", shape=(?, ?), dtype=float32)
x_poison_features:  Tensor("Gather:0", shape=(1, ?), dtype=float32)
t_target_features:  Tensor("Gather_1:0", shape=(1, ?), dtype=float32)
Lp:  Tensor("norm/Squeeze:0", shape=(), dtype=float32)
LP_gradient Tensor("strided_slice_1:0", shape=(268203,), dtype=float32)


In [6]:
with full_graph_inception.as_default():
    clean_inception_features = full_model_inception.generate_inception_features(data_sets.train, None)
with full_graph_inceptionResNet.as_default():
    clean_inceptionResNet_features = full_model_inceptionResNet.generate_inception_features(data_sets.train, None)

In [9]:
# 2. Find the results from the ./Experiment_results/Experiment_1/ folder
def parse_file_name(fn):
    fn = fn[:-4]
    lst = fn.split('_')
    test_idx = int(lst[-1])
    num_poisoned_training_points = int(lst[-3])
    Experiment_number = int(lst[1])
    contents_type = lst[2]
    method = lst[3]
    return {
        "test_idx":test_idx,
        "num_poisoned_training_points":num_poisoned_training_points,
        "Experiment_number":Experiment_number,
        "contents_type":contents_type,
        "method":method,
    }

source_dir = "Experiment_results/Experiment_1/"
file_names = !ls -tr Experiment_results/Experiment_1/
poisoned_images, poisoned_train_indices, target_test_indices = [], [], []
for i in range(0, len(file_names), 2):
    file_name = file_names[i]
    dataset_name = dataset_classes['name'].replace('-', '_')
    # only look at the results from the specified dataset
    if dataset_name in file_name:
        file = parse_file_name(file_name)
        if file["num_poisoned_training_points"] == num_to_perterb and file["Experiment_number"] == 1 and ((use_IF and file["method"] == 'IF') or (not use_IF and file["method"] == 'FC')):
            if file["contents_type"] == 'indices':
                poisoned_train_index = np.load(source_dir + file_name)
                poisoned_image = np.load(source_dir + file_names[i+1])
            else:
                poisoned_train_index = np.load(source_dir + file_names[i+1])
                poisoned_image = np.load(source_dir + file_name)
            target_test_index = file['test_idx']
            poisoned_images.append(poisoned_image)
            poisoned_train_indices.append(poisoned_train_index)
            target_test_indices.append(target_test_index)
        

In [24]:
# 3. Generate polluted inception features

polluted_inception_features = []
with full_graph_inception.as_default():
    for poison_image, poisoned_train_index in zip(poisoned_images, poisoned_train_indices):
        poisoned_dataset = DataSet(poison_image, data_sets.train.labels[poisoned_train_index])
        polluted_inception_feature = full_model_inception.generate_inception_features(poisoned_dataset, None)
        polluted_inception_features.append(polluted_inception_feature)
       
polluted_inceptionResNet_features = []
with full_graph_inceptionResNet.as_default():
    for poison_image, poisoned_train_index in zip(poisoned_images, poisoned_train_indices):
        poisoned_dataset = DataSet(poison_image, data_sets.train.labels[poisoned_train_index])
        polluted_inceptionResNet_feature = full_model_inceptionResNet.generate_inception_features(poisoned_dataset, None)
        polluted_inceptionResNet_features.append(polluted_inceptionResNet_feature)

In [30]:
# 4. Initialize the top models (Log Reg) firstly then compute the final results

C = 1.0 / (len(data_sets.train.x) * .001)  
if num_classes == 2:
    log_reg_model_inception = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                warm_start=True, #True
                max_iter=1000)
    log_reg_model_inceptionResNet = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                warm_start=True, #True
                max_iter=1000)
else:
    log_reg_model_inception = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                multi_class='multinomial',
                warm_start=True, #True
                max_iter=1000) 
    log_reg_model_inceptionResNet = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                multi_class='multinomial',
                warm_start=True, #True
                max_iter=1000)

logits_inception, logits_inceptionResNet = [], []
for polluted_inception_feature, polluted_inceptionResNet_feature, poisoned_train_index, target_test_index in zip(polluted_inception_features, polluted_inceptionResNet_features, poisoned_train_indices, target_test_indices):
    target_test_dataset = DataSet(np.copy(data_sets.test.x[[target_test_index]]), np.copy(data_sets.test.labels[[target_test_index]]))
    with full_graph_inception.as_default():
        target_test_inception_feature = full_model_inception.generate_inception_features(target_test_dataset, None)   
    X, Y = np.copy(clean_inception_features), np.copy(data_sets.train.labels)
    X[poisoned_train_index] = polluted_inception_feature
    log_reg_model_inception.fit(X, Y)    
    logits_inception.append(log_reg_model_inception.predict_proba(target_test_inception_feature)[0][int(data_sets.test.labels[target_test_index])])
    
    with full_graph_inceptionResNet.as_default():
        target_test_inceptionResNet_feature = full_model_inceptionResNet.generate_inception_features(target_test_dataset, None)   
    X, Y = np.copy(clean_inceptionResNet_features), np.copy(data_sets.train.labels)
    X[poisoned_train_index] = polluted_inceptionResNet_feature
    log_reg_model_inceptionResNet.fit(X, Y)    
    logits_inceptionResNet.append(log_reg_model_inceptionResNet.predict_proba(target_test_inceptionResNet_feature)[0][int(data_sets.test.labels[target_test_index])])
    


In [31]:
logits_inception

[0.026601219866971445,
 0.9835105062431875,
 0.025527452686357033,
 0.00986500607154539,
 0.010891309051715203,
 0.005563066879134992,
 0.7506454908263247]

In [32]:
logits_inceptionResNet

[0.9976384792791324,
 0.9997061453241939,
 0.9996707425811158,
 0.9996608672139135,
 0.9999209242376349,
 0.9979180232567562,
 0.9997762087366919]